In [3]:
import pygame
from pygame.locals import *
import sys
from geopy.geocoders import Nominatim
from PIL import Image
import random
import os
import time
import cv2
import geopy.distance
import numpy as np

#Load and resize Image
def resizeImage(weight, height):
    image = pygame.image.load(r'images/WorldMap.jpg') #load image
    image = pygame.transform.scale(image, (weight, height)) #resize image
    rect = image.get_rect() #create react
    rect = rect.move((0, 0)) #place the image
    return rect,image

#Convert Y to LAT 
# Calcul to obtain coefficients are on the excel file.
def getLat(Y,coef_Y_positive,coef_Y_negative):
    if(Y > 0):
        if(Y >= 0 and  Y <=76):
            LAT = Y/coef_Y_positive[0]
        elif(Y>77 and Y <=149):
            LAT = Y/coef_Y_positive[1]
        elif(Y>150 and Y <=211):
            LAT = Y/coef_Y_positive[2]
        elif(Y>212 and Y <=263):
            LAT = Y/coef_Y_positive[3]
        elif(Y>264 and Y <=288):
            LAT = Y/coef_Y_positive[4]
        elif(Y>289 and Y <=300):
            LAT = Y/coef_Y_positive[5]
        else:
            LAT = 0   
    else:
        if(Y > -84 and  Y <= 0):
            LAT = Y/coef_Y_negative[0]
        elif(Y > -154 and Y <= -85):
            LAT = Y/coef_Y_negative[1]
        elif(Y > -215 and Y <= -155):
            LAT = Y/coef_Y_negative[2]
        elif(Y > -265 and Y <= -216):
            LAT = Y/coef_Y_negative[3]
        elif(Y >= -300 and Y <= -266):
            LAT = Y/coef_Y_negative[4]
        else:
            LAT = 0
    return LAT


#Convert X to LAT 
def getLon(X,coef_X):
    return X/coef_X

#Convert the geolocalisation guess to country name
def getCountry(geolocalisation):
    geolocator = Nominatim(user_agent="http")#Initialisation
    location = geolocator.reverse(geolocalisation, language='en')#Convert geolocalisation to country
    if(location != None):#If found one
        adress = location.address.split(", ") #location address countain all informations so we split into list
        adress = adress[len(adress)-1] #The country name is at the last-1 position of the list
        adress = adress.lower() #Lower the uppercase
        return adress
    else:#No country found
        coo = "Nothing" 
        coo = coo.lower()
        return coo

#Get random image for the game
def getImage():
    path="./images/photos/" 
    files=os.listdir(path)#Go to images path and store all their name
    image = random.choice(files)#Take one random
    image_path = str(path) + str(image) #Obtain the path of the images choosen
    img = cv2.imread(image_path) #Read the timage
    cv2.imshow("PylandGuessr Image",img) #Show the image
    image = image.replace(".jpg", "") #Replace the .jpg to get only the country name
    image = image.lower() #Lower uppercase
    return image

# Get the direction of the country based on the location clicked by the user if it is wrong
def CardinalPoint(LAT,LatImage,LON,LonImage):
    cardinal = ["North","South","Est","West"]
    cardinal_score = [0,0,0,0]
    #If the latitude guess is larger than the latitude country add 1 to South
    if (LAT > LatImage and ((LAT-LatImage > 5) or (LAT-LatImage < -5))):
        cardinal_score[1] += 1
    #If the latitude guess is lower than the latitude country add 1 to North
    elif (LAT < LatImage and ((LAT-LatImage > 5) or (LAT-LatImage < -5))):
        cardinal_score[0] += 1
    #If the longitude guess is lower than the latitude country add 1 to Est
    if (LON < LonImage and ((LON-LonImage > 5) or (LON-LonImage < -5))):
        cardinal_score[2] += 1
    #If the longitude guess is larger than the latitude country add 1 to West
    elif (LON > LonImage and ((LON-LonImage > 5) or (LON-LonImage < -5))):
        cardinal_score[3] += 1
    directionSearch = " "
    for i in range(len(cardinal_score)):
        if(cardinal_score[i] == 1):
            directionSearch += " " + cardinal[i] #merge to get the direction (example : "SOUTH", "NORTH EST", "NORTH WEST", ...)
    return directionSearch

#Message display when the guess is wrong
def WrongAnswer(country,imageCountry, tryNumber,display,geolocalisation,LAT , LON, geolocalisationImage, LatImage, LonImage, weight, height):
    
    #calculate the distance between the guess and the country to guess (in km)
    distance = geopy.distance.geodesic(geolocalisation,geolocalisationImage).km
    distance = np.round(distance, 2)#Rounding
    
    directionSearch = CardinalPoint(LAT,LatImage,LON,LonImage)#Obtain the direction
    
    falseReponse = "%s"% (country)#Text to display country name
    Attempt = "%s attempts left"%(tryNumber)#Text to display number of attempt left
    dist = "%s km away, %s"%(distance,directionSearch)#Text to display distance and direction
    lenTexteFalse = len(falseReponse) #Obtain len
    lenAttempt = len(Attempt) #Obtain len
    lendist = len(dist) #Obtain len

    font = pygame.font.Font("freesansbold.ttf", 25)#Initialize
    textFalse = font.render(falseReponse,True,(255,0,0))#adapt and color the text
    textAttempt = font.render(Attempt,True,(255,0,0))#adapt and color the text
    textdist = font.render(dist,True,(255,0,0))#adapt and color the text

    display.blit(textFalse,((weight/2)-5*lenTexteFalse,height/2))#Display text at the center of the window
    display.blit(textAttempt,((weight/2)-5*lenAttempt,(height/2)+25)) #Display text at the center of the window but below the first
    display.blit(textdist,((weight/2)-5*lendist,(height/2)+50)) #Display text at the center of the windows but below the second
    pygame.display.update()#Update the window
    time.sleep(3) #Wait to allows user to see the text

# When where are no attempt left
def gameOver(country,display,weight, height):
    
    falseReponse = "%s"% (country) #Text to display country name
    lenTexteFalse = len(falseReponse) #Len text

    Loose = "You loose !" #Sentence to display
    lenLoose = len(Loose) #Len text

    font = pygame.font.Font("freesansbold.ttf", 25)#Initialize
    textFalse = font.render(falseReponse,True,(255,0,0))#Adapt and color the text
    textLoose = font.render(Loose,True,(255,0,0))#Adapt and color the text

    display.blit(textFalse,((weight/2)-5*lenTexteFalse,height/2))#Display text at the center of the window
    display.blit(textLoose,((weight/2)-5*lenLoose,(height/2)+25))#Display text at the center of the window but below the first
    pygame.display.update()#Update the window
    time.sleep(2)#Wait to allows user to see the text
    cv2.destroyAllWindows()#destroy all windows
    pygame.quit()
    sys.exit()

#When user find the good country
def GoodAnswer(country,display,weight,height):
    goodReponse = "%s"% (country)#Text to display country name
    Victory = "You found the country !!!"#Sentence to display
    lenTexteTrue = len(goodReponse)#Len text
    lenVictory = len(Victory)#Len text
    font = pygame.font.Font("freesansbold.ttf", 25)#Initialize
    textGood = font.render(goodReponse,True,(0,255,0))#Adapt and color the text
    textVictory = font.render(Victory,True,(0,255,0))#Adapt and color the text

    display.blit(textGood,((weight/2)-5*lenTexteTrue,height/2))#Display text at the center of the window
    display.blit(textVictory,((weight/2)-5*lenVictory,(height/2)+25))#Display text at the center of the window but below the first
    pygame.display.update()#Update the window
    time.sleep(2)#Wait to allows user to see the text
    cv2.destroyAllWindows()#destroy all windows
    pygame.quit()
    sys.exit()
    
#When the geolocalisation gound nothing
def noCountryFound(weight,height,display):
    
    badLocation = "Nothing was found here"#Sentence to display
    lenbadLocation = len(badLocation)#Len text

    font = pygame.font.Font("freesansbold.ttf", 25)#Initialize
    textNothing = font.render(badLocation,True,(255,0,0))#Adapt and color the text

    display.blit(textNothing,((weight/2)-5*lenbadLocation,height/2))#Display text at the center of the window
    pygame.display.update()#Update the window
    time.sleep(2)#Wait to allows user to see the text


def main():
    pygame.init()#Initalize pygame
    weight = 1200 #set weight
    height = 600 #set height
    display = pygame.display.set_mode((weight, height)) #initialize display
    FPS_CLOCK = pygame.time.Clock() #initialize fps
    rect,image = resizeImage(weight, height) #Obtain image
    
    #coefficient used for convert X and Y to Lon and Lat
    coef_Y_positive = [5.8,5.5,4.7,4.5,3.8,3.5]
    coef_Y_negative = [5.78,5.7,5.0,4.6,3.5]
    coef_X = 3.35
    
    #Get random image
    imageCountry = getImage()
    tryNumber = 5
    #Launch the game on the window
    while True:
        display.blit(image, rect)#Display image
        
        for event in pygame.event.get():#If click on [x] then destroy window and quit
            if event.type == QUIT:
                pygame.quit()
                cv2.destroyAllWindows()
                sys.exit()

            if event.type == pygame.MOUSEBUTTONDOWN: #If left click on mouse (prevent for long click)
                mouse_presses = pygame.mouse.get_pressed()#If left click on mouse
                if mouse_presses[0]:
                    coordinates = pygame.mouse.get_pos()#Obtain x and y of the mouse position on the window
                    coordinates = list(coordinates)#Convert to list
                    X = coordinates[0]#Get X
                    Y = coordinates[1]#Get Y
                    X = (X-(weight/2))#Convert X to set the 0 at the center of the image
                    Y = -(Y-(height/2))#Convert Y to set the 0 at the center of the image
                    
                    LAT = getLat(Y,coef_Y_positive,coef_Y_negative)#Obtain Latitude
                    LON = getLon(X,coef_X)#Obtain Longitude
                    geolocalisation = str(LAT)+","+str(LON)#Create geolocalisation
                    
                    country = getCountry(geolocalisation)#Obtain country name of the guess
                    
                    loc = Nominatim(user_agent="GetLoc")# calling the Nominatim tool

                    getLocImage = loc.geocode(imageCountry)# Obtain the geolocalisation from the country name of the image
                    LatImage = getLocImage.latitude #Obtain lat
                    LonImage = getLocImage.longitude #Obtain lon
                    geolocalisationImage = (str(LatImage) +","+ str(LonImage))#Create geolocalisation
                    
                    if(country!="nothing"):#If a country is guess
                        if(imageCountry==country):#If the country is correctly guess            
                            GoodAnswer(country,display,weight,height) #Display win
                        else:
                            tryNumber -= 1 # Attempt left decrease
                            if(tryNumber>0):#If where are attempt left display direction and distance
                                WrongAnswer(country,imageCountry, tryNumber,display,geolocalisation,LAT ,LON, geolocalisationImage,LatImage, LonImage ,weight, height)
                            else:#no attempt left display Game Over
                                gameOver(country,display,weight, height)

                    else:#Display no country found
                        noCountryFound(weight,height,display)
                        
        pygame.display.update()#Update the window
        FPS_CLOCK.tick(30)
        
if __name__ == "__main__":
    main()

SystemExit: 